In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
import math
seaborn.set(style='whitegrid')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inlineb
from datetime import datetime, date

C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#read in the baseline data
base = pd.read_csv("MID pre-injection baseline.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
base.drop([0,1], inplace=True)
print(base.describe())
#bring the time data into a new df
time1 = pd.DataFrame(base.iloc[:,0::2]) #- get every second column
#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,12,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time1)

#time["Elapsed time, mass 40"].describe()

       mass 2.0_value  mass 4.0_value  mass 12.0_value  mass 16.0_value  \
count    9.350000e+02    9.350000e+02     9.350000e+02     9.350000e+02   
mean     8.488531e-10    1.952450e-12     2.589657e-12     7.733964e-12   
std      1.590351e-09    8.179332e-13     1.983329e-12     9.201592e-12   
min      4.703610e-11    2.900000e-15     3.338000e-13     4.284000e-13   
25%      4.927365e-11    2.195550e-12     1.660300e-12     3.086350e-12   
50%      5.236590e-11    2.296500e-12     1.800100e-12     3.298600e-12   
75%      6.048245e-11    2.424500e-12     2.027900e-12     3.650300e-12   
max      5.963610e-09    5.195300e-12     9.802300e-12     3.687190e-11   

       mass 18.0_value  mass 28.0_value  mass 32.0_value  mass 40.0_value  \
count     9.350000e+02     9.350000e+02     9.350000e+02     9.350000e+02   
mean      4.177543e-12     2.945769e-11     3.807775e-13     7.135476e-12   
std       4.075348e-12     3.968642e-11     2.134044e-13     4.885076e-12   
min       2.1300

,mass 2.0_time,mass 4.0_time,mass 12.0_time,mass 16.0_time,mass 18.0_time,mass 28.0_time,mass 32.0_time,mass 40.0_time,mass 44.0_time,"Elapsed time, mass 2","Elapsed time, mass 4","Elapsed time, mass 12","Elapsed time, mass 16","Elapsed time, mass 18","Elapsed time, mass 28","Elapsed time, mass 32","Elapsed time, mass 40","Elapsed time, mass 44"
2,2022/11/30 15:07:20.301,2022/11/30 15:07:20.551,2022/11/30 15:07:20.801,2022/11/30 15:07:21.051,2022/11/30 15:07:21.301,2022/11/30 15:07:21.551,2022/11/30 15:07:21.802,2022/11/30 15:07:22.052,2022/11/30 15:07:22.302,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,2022/11/30 15:07:22.623,2022/11/30 15:07:22.873,2022/11/30 15:07:23.123,2022/11/30 15:07:23.374,2022/11/30 15:07:23.624,2022/11/30 15:07:23.874,2022/11/30 15:07:24.124,2022/11/30 15:07:24.625,NaN,2.322,2.322,2.322,2.323,2.323,2.323,2.322,2.573,NaN
4,2022/11/30 15:07:24.946,2022/11/30 15:07:25.196,2022/11/30 15:07:25.446,2022/11/30 15:07:25.696,2022/11/30 15:07:25.946,2022/11/30 15:07:26.197,2022/11/30 15:07:26.447,2022/11/30 15:07:26.697,2022/11/30 15:07:26.947,4.645,4.645,4.645,4.645,4.645,4.646,4.645,4.645,4.645
5,2022/11/30 15:07:27.268,2022/11/30 15:07:27.518,2022/11/30 15:07:27.769,2022/11/30 15:07:28.019,2022/11/30 15:07:28.269,2022/11/30 15:07:28.519,2022/11/30 15:07:28.769,2022/11/30 15:07:29.270,NaN,6.967,6.967,6.968,6.968,6.968,6.968,6.967,7.218,NaN
6,2022/11/30 15:07:29.591,2022/11/30 15:07:29.841,2022/11/30 15:07:30.091,2022/11/30 15:07:30.341,2022/11/30 15:07:30.591,2022/11/30 15:07:30.842,2022/11/30 15:07:31.092,2022/11/30 15:07:31.592,NaN,9.290,9.290,9.290,9.290,9.290,9.291,9.290,9.540,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932,2022/11/30 15:59:27.256,2022/11/30 15:59:30.507,2022/11/30 15:59:30.757,2022/11/30 15:59:31.007,2022/11/30 15:59:31.257,2022/11/30 15:59:32.008,2022/11/30 15:59:35.259,2022/11/30 15:59:35.509,2022/11/30 15:59:35.759,3126.955,3129.956,3129.956,3129.956,3129.956,3130.457,3133.457,3133.457,3133.457
933,2022/11/30 15:59:36.581,2022/11/30 15:59:39.832,2022/11/30 15:59:40.082,2022/11/30 15:59:40.332,2022/11/30 15:59:40.582,2022/11/30 15:59:41.333,2022/11/30 15:59:44.584,2022/11/30 15:59:44.834,2022/11/30 15:59:45.084,3136.280,3139.281,3139.281,3139.281,3139.281,3139.782,3142.782,3142.782,3142.782
934,2022/11/30 15:59:45.906,2022/11/30 15:59:49.157,2022/11/30 15:59:49.407,2022/11/30 15:59:49.657,2022/11/30 15:59:49.907,2022/11/30 15:59:50.658,2022/11/30 15:59:53.909,2022/11/30 15:59:54.159,2022/11/30 15:59:54.409,3145.605,3148.606,3148.606,3148.606,3148.606,3149.107,3152.107,3152.107,3152.107
935,2022/11/30 15:59:55.231,2022/11/30 15:59:58.482,2022/11/30 15:59:58.732,2022/11/30 15:59:58.982,2022/11/30 15:59:59.232,2022/11/30 15:59:59.983,2022/11/30 16:00:03.234,2022/11/30 16:00:03.484,2022/11/30 16:00:03.734,3154.930,3157.931,3157.931,3157.931,3157.931,3158.432,3161.432,3161.432,3161.432


In [3]:
#read in the measurement data
mid = pd.read_csv("MID experiment log.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True)
print(mid.describe())
#bring the time data into a new df
time2 = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,12,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time2)

#time["Elapsed time, mass 40"].describe()

       mass 2.0_value  mass 4.0_value  mass 12.0_value  mass 16.0_value  \
count    5.510000e+02    5.510000e+02     5.510000e+02     5.510000e+02   
mean     3.225355e-10    7.265162e-13     2.172243e-12     4.894407e-12   
std      3.681205e-11    1.277909e-13     5.422036e-13     5.588653e-13   
min      2.651810e-10    1.000000e-16     7.272000e-13     2.242800e-12   
25%      2.916230e-10    6.778000e-13     1.958000e-12     4.531750e-12   
50%      3.189090e-10    7.212000e-13     2.116900e-12     4.839600e-12   
75%      3.521835e-10    7.808000e-13     2.342600e-12     5.279900e-12   
max      3.919720e-10    2.217500e-12     1.063340e-11     6.108300e-12   

       mass 18.0_value  mass 28.0_value  mass 32.0_value  mass 40.0_value  \
count     5.510000e+02     5.510000e+02     5.510000e+02     1.300000e+01   
mean      8.344284e-12     5.088958e-13     2.065205e-12     2.184669e-12   
std       1.163134e-12     1.144364e-12     3.748639e-13     1.180252e-12   
min       5.0767

,mass 2.0_time,mass 4.0_time,mass 12.0_time,mass 16.0_time,mass 18.0_time,mass 28.0_time,mass 32.0_time,mass 40.0_time,mass 44.0_time,"Elapsed time, mass 2","Elapsed time, mass 4","Elapsed time, mass 12","Elapsed time, mass 16","Elapsed time, mass 18","Elapsed time, mass 28","Elapsed time, mass 32","Elapsed time, mass 40","Elapsed time, mass 44"
2,2022/11/30 17:47:50.741,2022/11/30 17:47:53.992,2022/11/30 17:47:54.242,2022/11/30 17:47:54.492,2022/11/30 17:47:54.743,2022/11/30 17:47:54.993,2022/11/30 17:47:55.243,2022/11/30 17:47:55.493,2022/11/30 17:47:55.743,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.00
3,2022/11/30 17:47:56.636,2022/11/30 17:48:00.137,2022/11/30 17:48:00.387,2022/11/30 17:48:00.638,2022/11/30 17:48:00.888,2022/11/30 17:48:01.138,2022/11/30 17:48:01.638,NaN,NaN,5.895,6.145,6.145,6.146,6.145,6.145,6.395,NaN,NaN
4,2022/11/30 17:48:02.531,2022/11/30 17:48:05.782,2022/11/30 17:48:06.032,2022/11/30 17:48:06.282,2022/11/30 17:48:06.533,2022/11/30 17:48:06.783,2022/11/30 17:48:07.033,2022/11/30 17:48:07.283,2022/11/30 17:48:07.533,11.790,11.790,11.790,11.790,11.790,11.790,11.790,11.79,11.79
5,2022/11/30 17:48:08.427,2022/11/30 17:48:11.928,2022/11/30 17:48:12.178,2022/11/30 17:48:12.428,2022/11/30 17:48:12.678,2022/11/30 17:48:12.929,2022/11/30 17:48:13.429,NaN,NaN,17.686,17.936,17.936,17.936,17.935,17.936,18.186,NaN,NaN
6,2022/11/30 17:48:14.322,2022/11/30 17:48:17.823,2022/11/30 17:48:18.073,2022/11/30 17:48:18.323,2022/11/30 17:48:18.573,2022/11/30 17:48:18.824,2022/11/30 17:48:19.324,NaN,NaN,23.581,23.831,23.831,23.831,23.830,23.831,24.081,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,2022/11/30 18:41:29.412,2022/11/30 18:41:32.913,2022/11/30 18:41:33.163,2022/11/30 18:41:33.413,2022/11/30 18:41:33.663,2022/11/30 18:41:33.914,2022/11/30 18:41:34.414,NaN,NaN,3218.671,3218.921,3218.921,3218.921,3218.920,3218.921,3219.171,NaN,NaN
549,2022/11/30 18:41:35.307,2022/11/30 18:41:38.808,2022/11/30 18:41:39.058,2022/11/30 18:41:39.308,2022/11/30 18:41:39.558,2022/11/30 18:41:39.809,2022/11/30 18:41:40.309,NaN,NaN,3224.566,3224.816,3224.816,3224.816,3224.815,3224.816,3225.066,NaN,NaN
550,2022/11/30 18:41:41.202,2022/11/30 18:41:44.703,2022/11/30 18:41:44.953,2022/11/30 18:41:45.203,2022/11/30 18:41:45.453,2022/11/30 18:41:45.704,2022/11/30 18:41:46.204,NaN,NaN,3230.461,3230.711,3230.711,3230.711,3230.710,3230.711,3230.961,NaN,NaN
551,2022/11/30 18:41:47.097,2022/11/30 18:41:50.598,2022/11/30 18:41:50.848,2022/11/30 18:41:51.098,2022/11/30 18:41:51.348,2022/11/30 18:41:51.599,2022/11/30 18:41:52.099,NaN,NaN,3236.356,3236.606,3236.606,3236.606,3236.605,3236.606,3236.856,NaN,NaN


In [4]:
#reading the vaclog
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
1,NaN,30/11/2022 14:56:15,7.0,0.50226,5.320000e-08,5.240000e-08,2.828000e-08,6.246000e-07,0.000616,0.0,-32.099,164.758,NaN,NaN
2,NaN,30/11/2022 14:56:22,14.0,0.50199,5.350000e-08,5.280000e-08,2.842000e-08,6.246000e-07,0.000607,0.0,-32.435,164.646,NaN,NaN
3,NaN,30/11/2022 14:56:29,21.0,0.49881,5.390000e-08,5.310000e-08,2.866000e-08,6.238000e-07,0.000612,0.0,-32.435,164.758,NaN,NaN
4,NaN,30/11/2022 14:56:37,29.0,0.50170,5.420000e-08,5.350000e-08,2.886000e-08,6.246000e-07,0.000612,0.0,-32.323,164.758,NaN,NaN
5,NaN,30/11/2022 14:56:44,36.0,0.50212,5.460000e-08,5.390000e-08,2.911000e-08,6.246000e-07,0.000635,0.0,-32.323,164.646,NaN,NaN


In [5]:
#Reading in the HiVolta measurement file
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["em"]=abs(hv["em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding an elapsed time column
timestamp=[]
for i in hv["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)
hv.drop(index=hv.index[0], 
        axis=0, 
        inplace=True)

hv.describe()



,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,VMon8,IMon1,IMon2,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,hv_grid,em
count,1638.000000,1638.000000,1638.000000,1638.000000,1638.000000,1638.000000,1638.0,1638.000000,1.638000e+03,1638.000000,1638.000000,1638.000000,1638.000000,1638.000000,1638.000000,1638.000000,1638.000000,1.638000e+03,1.638000e+03
mean,1062.690707,156.071099,97.957973,974.290672,973.056190,971.390183,0.0,0.007692,4.000000e-02,-0.094026,0.720994,12.269118,12.282255,12.255020,-0.010336,-0.017934,-0.003385,1.097325e-07,7.210082e-07
std,568.602272,230.724874,14.309678,156.534537,160.312735,164.725834,0.0,0.009733,5.830451e-16,0.093527,0.518559,1.994183,2.004024,2.025827,0.000771,0.001130,0.000798,7.446494e-08,5.185388e-07
min,1.190000,0.040000,0.040000,0.000000,0.000000,0.000000,0.0,0.000000,4.000000e-02,-0.611200,-0.001000,-0.042200,-0.031700,-0.049600,-0.012100,-0.020100,-0.004200,0.000000e+00,3.000000e-10
25%,570.845250,0.120000,100.060000,1000.020000,1000.200000,1000.180000,0.0,0.000000,4.000000e-02,-0.155300,0.014100,12.434800,12.448750,12.429650,-0.010900,-0.018800,-0.004100,1.740000e-08,1.410000e-08
50%,1062.730000,0.120000,100.060000,1000.060000,1000.260000,1000.240000,0.0,0.000000,4.000000e-02,-0.142300,0.996900,12.607550,12.623550,12.605600,-0.010200,-0.018500,-0.003600,1.425000e-07,9.969000e-07
75%,1554.640000,499.940000,100.080000,1000.080000,1000.260000,1000.240000,0.0,0.020000,4.000000e-02,0.015700,1.109175,12.756300,12.773025,12.752875,-0.010000,-0.016700,-0.003000,1.554750e-07,1.109175e-06
max,2046.131000,500.000000,100.140000,1000.100000,1000.260000,1000.300000,0.0,0.020000,4.000000e-02,0.514000,2.964400,13.495700,13.545200,13.500200,-0.008600,-0.016000,-0.001400,6.112000e-07,2.964400e-06


### Data processing

In [6]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [7]:
#Filtering background signal


#remove values under noise level from MID spectrum
mid["mass 40.0_value"][mid["mass 40.0_value"] < 1e-14] = np.nan
base["mass 40.0_value"][base["mass 40.0_value"] < 1e-14] = np.nan



C:\Users\etiirine\AppData\Local\Temp\ipykernel_18604\162514978.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mid["mass 40.0_value"][mid["mass 40.0_value"] < 1e-14] = np.nan
C:\Users\etiirine\AppData\Local\Temp\ipykernel_18604\162514978.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base["mass 40.0_value"][base["mass 40.0_value"] < 1e-14] = np.nan


### Plotting

In [8]:
#Plotting baseline MID scan
plt.figure(figsize=(14,8))
plt.plot(time1['Elapsed time, mass 2'],base["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(time1['Elapsed time, mass 4'],base["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(time1['Elapsed time, mass 12'],base["mass 12.0_value"],marker=".", markersize=4,label='M14')
plt.plot(time1['Elapsed time, mass 16'],base["mass 16.0_value"],marker=".", markersize=4,label='M16')
plt.plot(time1['Elapsed time, mass 18'],base["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(time1['Elapsed time, mass 28'],base["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(time1['Elapsed time, mass 32'],base["mass 32.0_value"],marker=".", markersize=4,label='M12')
plt.plot(time1['Elapsed time, mass 40'],base["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(time1["Elapsed time, mass 44"],base["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("Baseline MID @4K EGA on a-C with pre-injection of 1 ML")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 12,
        }
plt.text(400, 8e-11, '\u2190 EGA moved inside', fontdict = font)
plt.text(1200, 3.35e-10, 'Static vacuum \u2192', fontdict = font)

plt.yscale('log')

In [9]:
#Plotting experiment MID scan
plt.figure(figsize=(14,8))
plt.plot(time2['Elapsed time, mass 2'],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(time2['Elapsed time, mass 4'],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(time2['Elapsed time, mass 12'],mid["mass 12.0_value"],marker=".", markersize=4,label='M14')
plt.plot(time2['Elapsed time, mass 16'],mid["mass 16.0_value"],marker=".", markersize=4,label='M16')
plt.plot(time2['Elapsed time, mass 18'],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(time2['Elapsed time, mass 28'],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(time2['Elapsed time, mass 32'],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(time2['Elapsed time, mass 40'],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(time2["Elapsed time, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("MID during experiment @4K EGA on a-C with pre-injection of 1 ML")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 12,
        }
plt.text(12, 1.56e-10, 'Instruments on, base pressure ~stable', fontdict = font)
plt.text(1050, 1.56e-10, '\u2191 HiVolta log started', fontdict = font)
plt.text(2450, 3.35e-10, 'EGA turned', fontdict = font)

plt.yscale('log')

In [10]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution @4K EGA on a-C with pre-injection of 1 ML")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 12,
        }
plt.text(200, 1.4e-6, "First emission", fontdict = font)
plt.text(880, 6.3e-9, 'Grid repelling', fontdict = font)
plt.text(1100, 1.6e-6, 'Grid transparent', fontdict = font)
plt.text(1340, 6.3e-9, 'Grid repelling, EGA turned', fontdict = font)

plt.yscale('log')


In [11]:
#Plotting total pressures
plt.figure(figsize=(10,6))
plt.plot(vaclog["Elapsed time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.plot(vaclog["Elapsed time"],vaclog["DUAL experiment"],marker=".", markersize=4,label='F-R')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Total pressure evolution @4K EGA on a-C with pre-injection of 1 ML")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 12,
        }
plt.text(-440, 1.08e-7, 'Static vacuum', fontdict = font)
plt.text(2500, 9.4e-9, '\u2190 Dynamic vac', fontdict = font)
plt.text(560, 1.08e-9, 'EGA moved', fontdict = font)
plt.text(2130, 2.18e-7, 'Static vacuum', fontdict = font)
plt.text(4550, 1.1e-9, 'BA 1,2 off', fontdict = font)
plt.text(5500, 2e-4, '\u2190 Injection complete', fontdict = font)
plt.text(5300, 1.2e-8, 'LHe fill', fontdict = font)
plt.text(6950, 3.6e-7, '\u2190 Instruments on', fontdict = font)
plt.text(10540, 2e-8, 'EGA turned \u2192', fontdict = font)

ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()

In [12]:
#Plotting the temp evolution
plt.figure()
plt.plot((vaclog["Elapsed time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='Temperature log during the experiment')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Temperature log, EGA on a-C with pre-injection of 1 ML ")
plt.yscale('linear')